# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
# Import API key
from api_keys import g_key
gmaps.configure(api_key = g_key)
from citipy import citipy

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather = pd.read_csv("../WeatherPy/weather/weather.csv")
weather_data_df = weather.drop(columns=["Unnamed: 0"])
weather_data_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,celestun,1,MX,1609818467,89,20.87,-90.40,72.59,11.97
1,chokurdakh,55,RU,1609818467,85,70.63,147.92,-25.11,11.36
2,new norfolk,98,LR,1609818191,94,6.65,-10.59,74.88,1.25
3,rikitea,9,PF,1609818467,73,-23.12,-134.97,76.93,17.02
4,mataura,100,NZ,1609818467,87,-46.19,168.86,58.37,0.89
...,...,...,...,...,...,...,...,...,...
558,novoukrainskoye,99,RU,1609818513,88,45.38,40.53,34.54,11.30
559,porto novo,0,PT,1609818513,100,41.15,-8.61,33.80,4.74
560,susanville,90,US,1609818513,86,40.42,-120.65,37.40,9.17
561,warrington,0,GB,1609818513,80,53.39,-2.58,37.00,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = weather_data_df[["Lat", "Lng"]]
humidity = weather_data_df["Humidity"].astype(float)
fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=5,
                                   point_radius=1)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#given conditions are:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
new_weather = pd.DataFrame(weather_data_df, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])

max_temp = (new_weather["Max Temp"] <= 80) & (new_weather["Max Temp"] > 70)
wind_speed = new_weather["Wind Speed"] < 10
cloudiness = new_weather["Cloudiness"] == 0

new_weather[max_temp & wind_speed & cloudiness]

,City,Max Temp,Wind Speed,Cloudiness
61,youkounkoun,70.25,2.28,0
95,collie,79.00,4.00,0
98,cayenne,75.20,2.24,0
106,lafiagi,73.09,3.85,0
157,velingara,74.26,6.60,0
200,arlit,70.79,4.45,0
218,caravelas,75.99,7.45,0
245,manacapuru,77.00,3.49,0
264,bubaque,79.72,7.20,0
271,zhuhai,72.00,9.17,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = pd.DataFrame(weather, columns=["City", "Country", "Lat", "Lng"])
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
0,celestun,MX,20.87,-90.40,
1,chokurdakh,RU,70.63,147.92,
2,new norfolk,LR,6.65,-10.59,
3,rikitea,PF,-23.12,-134.97,
4,mataura,NZ,-46.19,168.86,


In [6]:
target_coordinates = f"{hotel_df['Lat'][0]},{hotel_df['Lng'][0]}"
target_search = "hotel"
target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using params dictionary
response = requests.get(base_url, params=params)

pprint(response.json(), depth=1)

{'html_attributions': [], 'results': [...], 'status': 'OK'}


In [7]:
#time to put the hotel names into a list
hotel_name = []

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to a json 
    name_address = name_address.json()
    try:
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)

In [8]:
#pain
hotel_name

['Posada Lilia',
 nan,
 nan,
 "Pension Maro'i",
 "Ellie's Villa",
 'Misty Waves Boutique Hotel',
 'Paradise Inn',
 'Observatory Guest House',
 'Desert Gardens Hotel - Ayers Rock Resort',
 'Sheraton Kauai Resort at Coconut Beach',
 'Talnakh Hotel',
 'Hotel Morón',
 nan,
 'Pensión Residencial Victoria Drinks&Food',
 'Nuli Sapi',
 'Sandpiper Motel',
 'Villa Enata',
 'Scoop Guest House',
 'Siang Guest House',
 'Villa del Palmar Flamingos Beach Resort & Spa Riviera Nayarit',
 'Motel 6 Cincinnati, OH - Central - Norwood',
 'Kaitangata Motor Camp',
 "Marriott's Ko Olina Beach Club",
 'Radisson Blu Polar Hotel, Spitsbergen',
 'Chambres d\'hôte "La Trinité"',
 'Vyjannex boarding house',
 'Hengifoss Guesthouse',
 'Keleti Beach Resort',
 'Hotel Peace Land Ishigakijima',
 nan,
 'Albatros Hotel',
 'Holiday Inn Express Lompoc',
 'Hotel Anastasia',
 'Qaanaaq Hotel',
 'Broadwater Mariner Resort',
 'Burntwood Hotel',
 'Centre Pastoral St. Augustin',
 'Pondok Wisata Elim',
 'Hilo Hawaiian Hotel',
 'Fuen

In [9]:
#insert hotel names into the DF and drop NA's
hotel_df['Hotel Name']= hotel_name
hotel_df.dropna()
#export a csv file
hotel_df.to_csv('hotel.csv')

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))